# Caracteristica MOD11A2 LST google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap = 'SATELLITE')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"}, name = "Zona")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Coleccion MODIS11 A2

In [6]:
# Coleccion de MODIS LST
MODIS_LST = ee.ImageCollection("MODIS/006/MOD11A2")\
              .filterDate('2019-01-01','2019-12-31')\
              .filterBounds(zona_ee)

In [8]:
# Conteo de imagenes
count = MODIS_LST.size().getInfo()
print("Cantidad de imagenes MODIS_LST:", count)

Cantidad de imagenes MODIS_LST: 46


In [9]:
# Imprimir la lista de ID Imagenes
ID_MODIS_LST = MODIS_LST.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_MODIS_LST)

['2019_01_01', '2019_01_09', '2019_01_17', '2019_01_25', '2019_02_02', '2019_02_10', '2019_02_18', '2019_02_26', '2019_03_06', '2019_03_14', '2019_03_22', '2019_03_30', '2019_04_07', '2019_04_15', '2019_04_23', '2019_05_01', '2019_05_09', '2019_05_17', '2019_05_25', '2019_06_02', '2019_06_10', '2019_06_18', '2019_06_26', '2019_07_04', '2019_07_12', '2019_07_20', '2019_07_28', '2019_08_05', '2019_08_13', '2019_08_21', '2019_08_29', '2019_09_06', '2019_09_14', '2019_09_22', '2019_09_30', '2019_10_08', '2019_10_16', '2019_10_24', '2019_11_01', '2019_11_09', '2019_11_17', '2019_11_25', '2019_12_03', '2019_12_11', '2019_12_19', '2019_12_27']


## Seleccion de una imagen

In [10]:
ID_MODIS_LST[26]

'2019_07_28'

In [12]:
# Importar imagen MODIS A2 LST
MODIS_img = ee.Image("MODIS/006/MOD11A2" + "/" + ID_MODIS_LST[26])

In [13]:
# Nombre de las bandas
print(MODIS_img.bandNames().getInfo())

['LST_Day_1km', 'QC_Day', 'Day_view_time', 'Day_view_angl', 'LST_Night_1km', 'QC_Night', 'Night_view_time', 'Night_view_angl', 'Emis_31', 'Emis_32', 'Clear_sky_days', 'Clear_sky_nights']


In [14]:
# Renombrar nombre de las bandas
MODIS_img = MODIS_img.select("LST_Day_1km")
MODIS_img = MODIS_img.rename("LST")

In [15]:
# Nombre de las bandas
print(MODIS_img.bandNames().getInfo())

['LST']


## Reproyectar

In [16]:
# Reproyectar segun zona de estudio
MODIS_img_py = MODIS_img.reproject(crs = "EPSG:4326", scale = 1000)

In [17]:
# Consultar Proyeccion
MODIS_img_py.projection().getInfo()

{'type': 'Projection',
 'crs': 'EPSG:4326',
 'transform': [0.008983152841195215, 0, 0, 0, -0.008983152841195215, 0]}

## Factor de escala

In [18]:
# Multiplicar factor de escala
MODIS_img_py1 = MODIS_img_py.multiply(0.02).subtract(273.15)

In [19]:
# Simbologia de MODIS
viz = {
    'min': 5,
    'max': 40,
    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003']
}

In [20]:
# Visualizar 
Map.addLayer(MODIS_img_py1, viz, name = "MODIS LST")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Recorte zona estudio

In [21]:
# Recortar zona estudio
MODIS_img_py1_clip = MODIS_img_py1.clip(zona_ee)

In [23]:
# Visualizar 
Map.addLayer(MODIS_img_py1_clip, viz,name = "MODIS LST clip")
Map

Map(bottom=4661.0, center=[-11.566143767762856, -66.81884765625001], controls=(WidgetControl(options=['positio…

## Reproyeccion UTM

In [24]:
# Cambiar la proyeccion
MODIS_img_py1_clip_utm = MODIS_img_py1_clip.reproject(crs = "EPSG:32719", scale = 1000)

## Exportar Imagen calibrado

In [25]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [26]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [27]:
# Exportar imagen directorio
geemap.ee_export_image(MODIS_img_py1_clip_utm, filename="MODIS_LST_clip_utm.tif", scale=1000, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\MODIS_LST_clip_utm.tif


In [28]:
# Exportar en google drive
geemap.ee_export_image_to_drive(MODIS_img_py1_clip_utm, description='MODIS_LST_clip_utm', folder='GEE_Python', region=geometria, scale=1000)

Exporting MODIS_LST_clip_utm ...
